In [1]:
# Basic imports
import numpy as np
import pandas as pd
import datetime # manipulating date formats
import itertools
import time
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error as MSE, r2_score, mean_absolute_percentage_error as MAPE

# Stats packages
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose,STL
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.tools import diff
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("imputed_df.csv", index_col=0,parse_dates=[0])
df.head()

,"(100, 'Blocks of flats, one-room flat')","(100, 'Blocks of flats, two-room flat')","(100, 'Blocks of flats, three-room flat+')","(100, 'Blocks of flats total')","(100, 'Building types total')","(120, 'Blocks of flats, one-room flat')","(120, 'Blocks of flats, two-room flat')","(120, 'Blocks of flats, three-room flat+')","(120, 'Blocks of flats total')","(120, 'Building types total')",...,"(96200, 'Building types total')","(96300, 'Blocks of flats, two-room flat')","(96300, 'Building types total')","(96400, 'Building types total')","(96440, 'Building types total')","(96460, 'Building types total')","(96500, 'Building types total')","(96900, 'Building types total')","(96910, 'Building types total')","(99600, 'Building types total')"
2010-04-01,5347,5021,5396,5219,5219,5646,5355,5713,5495,5495,...,1646,1525,1475,1499,1109,1568,1039,2030,1404,1059
2010-07-01,5826,5081,4828,5181,5181,5395,5534,5884,5595,5595,...,1668,1794,1782,1614,1069,1568,1069,1519,1354,836
2010-10-01,5566,5006,5394,5269,5269,5613,5321,5666,5521,5521,...,1619,1670,1750,1826,929,1568,1028,1839,1462,972
2011-01-01,5545,5713,5571,5632,5632,5905,5528,5691,5689,5689,...,1631,1501,1678,1697,1195,1615,1146,1868,1445,737
2011-04-01,5812,5866,5709,5807,5807,6332,5523,5571,5749,5749,...,1906,1713,1672,1565,932,1657,1097,1729,1544,932


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 45 entries, 2010-04-01 to 2021-04-01
Columns: 1193 entries, (100, 'Blocks of flats, one-room flat') to (99600, 'Building types total')
dtypes: int64(1193)
memory usage: 419.8 KB


In [3]:
def params_search(tgt_dict, named_ts, max_p=2, d=1, max_q=2, max_P=2, D=1, max_Q=2, s=4):
    """
    Keyword arguments:
    multivar_ts -- pandas dataframe of shape (n,m) where n is the number of
    time steps and n is the number of variables for prediction.
    """    
    # set parameter range
    col_name, ts = named_ts
    p,d,q = range(0,max_p+1),[d],range(0,max_q+1)
    P,D,Q,s = range(0,max_P+1),[D],range(0,max_Q+1),[s]
    
    # list of all parameter combos
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = list(itertools.product(P, D, Q, s))
    all_param = list(itertools.product(pdq,seasonal_pdq))

    best_param, best_aic = all_param[0], np.inf
    for param in all_param:
        try:
            mod = SARIMAX(
                ts,
                order=param[0],
                seasonal_order=param[1]
            )
            res = mod.fit(disp=0)
            if res.aic < best_aic:
                best_param, best_aic = param, res.aic
        except Exception as e:
            print(e)
            continue
    
    print(f'The best model for {col_name} is: SARIMAX{best_param[0]}x{best_param[1]} \
        - AIC:{round(best_aic,2)}')

    tgt_dict[col_name] = best_param
            
    return best_param

In [5]:
def df_params_search(df, max_p=2, d=1, max_q=2, max_P=2, D=1, max_Q=2, s=4):
    """
    Keyword arguments:
    multivar_ts -- pandas dataframe of shape (n,m) where n is the number of
    time steps and n is the number of variables for prediction.
    """
    res_df = pd.DataFrame(columns=df.columns, index=list('pdqPDQs'))
    # set parameter range
    p,d,q = range(0,max_p+1),[d],range(0,max_q+1)
    P,D,Q,s = range(0,max_P+1),[D],range(0,max_Q+1),[s]

    # list of all parameter combos
    pdq = list(map(list,itertools.product(p, d, q)))
    seasonal_pdq = list(map(list, itertools.product(P, D, Q, s)))
    all_param = list(map(list, itertools.product(pdq,seasonal_pdq)))

    for column in df:
        best_param, best_aic = [[0,0,0],[0,0,0,4]], np.inf
        for param in all_param:
            try:
                mod = SARIMAX(
                    df[column].values,
                    order=param[0],
                    seasonal_order=param[1]
                )
                
                res = mod.fit(disp=0)
                if res.aic < best_aic:
                    best_param, best_aic = param, res.aic
            except Exception as e:
                print(e)
                continue

        res_df[column] = best_param[0] + best_param[1]
    
        print(f'The best model for {column} is: SARIMAX{best_param[0]}x{best_param[1]} \
            - AIC:{round(best_aic,2)}')
            
    return res_df

In [7]:
res_df = df_params_search(df.iloc[:,:2])
res_df

The best model for (100, 'Blocks of flats, one-room flat') is: SARIMAX[0, 1, 1]x[0, 1, 1, 4]             - AIC:641.14
The best model for (100, 'Blocks of flats, two-room flat') is: SARIMAX[0, 1, 1]x[0, 1, 1, 4]             - AIC:591.47


In [8]:
res_df

,"(100, 'Blocks of flats, one-room flat')","(100, 'Blocks of flats, two-room flat')"
p,0,0
d,1,1
q,1,1
P,0,0
D,1,1
Q,1,1
s,4,4


In [104]:
from multiprocessing import Pool

def parallelize_dataframe(df, func, n_cores=12):
    df_split = np.array_split(df, n_cores, axis=1)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [105]:
df = parallelize_dataframe(df.iloc[:,:2], df_params_search)
df.head()

LU decomposition error.
The best model for (130, 'Building types total') is: SARIMAX[2, 1, 0]x[2, 1, 1, 4]             - AIC:637.64
LU decomposition error.
The best model for (150, 'Blocks of flats total') is: SARIMAX[1, 1, 0]x[0, 1, 1, 4]             - AIC:551.24
The best model for (200, 'Blocks of flats, two-room flat') is: SARIMAX[1, 1, 0]x[0, 1, 1, 4]             - AIC:526.31
The best model for (160, 'Blocks of flats total') is: SARIMAX[0, 1, 2]x[0, 1, 1, 4]             - AIC:596.98
The best model for (140, 'Blocks of flats, three-room flat+') is: SARIMAX[0, 1, 1]x[0, 1, 1, 4]             - AIC:610.12
LU decomposition error.
The best model for (100, 'Blocks of flats, one-room flat') is: SARIMAX[0, 1, 1]x[0, 1, 1, 4]             - AIC:641.14
The best model for (150, 'Blocks of flats, one-room flat') is: SARIMAX[1, 1, 0]x[0, 1, 1, 4]             - AIC:584.15
The best model for (120, 'Blocks of flats total') is: SARIMAX[0, 1, 1]x[0, 1, 1, 4]             - AIC:571.1
The best model for 

,"(100, 'Blocks of flats, one-room flat')","(100, 'Blocks of flats, two-room flat')","(100, 'Blocks of flats, three-room flat+')","(100, 'Blocks of flats total')","(100, 'Building types total')","(120, 'Blocks of flats, one-room flat')","(120, 'Blocks of flats, two-room flat')","(120, 'Blocks of flats, three-room flat+')","(120, 'Blocks of flats total')","(120, 'Building types total')",...,"(170, 'Blocks of flats total')","(170, 'Building types total')","(180, 'Blocks of flats, one-room flat')","(180, 'Blocks of flats, two-room flat')","(180, 'Blocks of flats, three-room flat+')","(180, 'Blocks of flats total')","(180, 'Building types total')","(200, 'Blocks of flats, two-room flat')","(200, 'Blocks of flats, three-room flat+')","(200, 'Building types total')"
p,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
d,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
q,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
df_params_search(df.iloc[:,:2])

The best model for (100, 'Blocks of flats, one-room flat') is: SARIMAX[0, 1, 1]x[0, 1, 1, 4]             - AIC:641.14
The best model for (100, 'Blocks of flats, two-room flat') is: SARIMAX[0, 1, 1]x[0, 1, 1, 4]             - AIC:591.47


,"(100, 'Blocks of flats, one-room flat')","(100, 'Blocks of flats, two-room flat')"
p,0,0
d,1,1
q,1,1
P,0,0
D,1,1
Q,1,1
s,4,4


In [9]:
import multiprocessing as mp
import pprint
from multiprocessing import Pool
from itertools import repeat

res = dict()

if __name__ == '__main__':
    with mp.Manager() as manager:
        d = manager.dict()
        named_ts = df.iloc[:,:38].to_dict(orient='list').items()
        args = list(zip(repeat(d, len(named_ts)), named_ts))
        with manager.Pool() as pool:
            pool.starmap(params_search, args)
        # `d` is a DictProxy object that can be converted to dict
        # pprint.pprint(dict(d))
        res = dict(d)
        pprint.pprint(res)

LU decomposition error.
The best model for (130, 'Building types total') is: SARIMAX(2, 1, 0)x(2, 1, 1, 4)         - AIC:637.64
The best model for (130, 'Blocks of flats total') is: SARIMAX(2, 1, 0)x(2, 1, 1, 4)         - AIC:637.64
The best model for (100, 'Blocks of flats, three-room flat+') is: SARIMAX(1, 1, 0)x(0, 1, 1, 4)         - AIC:588.76
LU decomposition error.
The best model for (120, 'Building types total') is: SARIMAX(1, 1, 0)x(0, 1, 1, 4)         - AIC:575.08
The best model for (100, 'Blocks of flats, two-room flat') is: SARIMAX(0, 1, 1)x(0, 1, 1, 4)         - AIC:591.47
LU decomposition error.
LU decomposition error.
The best model for (120, 'Blocks of flats, three-room flat+') is: SARIMAX(0, 1, 1)x(0, 1, 1, 4)         - AIC:601.39
The best model for (100, 'Blocks of flats, one-room flat') is: SARIMAX(0, 1, 1)x(0, 1, 1, 4)         - AIC:641.14
The best model for (120, 'Blocks of flats, one-room flat') is: SARIMAX(0, 1, 1)x(0, 1, 1, 4)         - AIC:603.71
The best model 

In [60]:
res_df = pd.DataFrame(res).T
pdq = pd.DataFrame(res_df.iloc[:,0].tolist(), index=res_df.index,columns=['p','d','q'])
PDQs = pd.DataFrame(res_df.iloc[:,1].tolist(), index=res_df.index,columns=['P','D','Q','s'])

res_df = pd.concat([pdq,PDQs], axis=1)
res_df.to_csv('res_df.csv',index=True)
res_df.head()

,p,d,q,P,D,Q,s
"(130, 'Building types total')",2,1,0,2,1,1,4
"(130, 'Blocks of flats total')",2,1,0,2,1,1,4
"(100, 'Blocks of flats, three-room flat+')",1,1,0,0,1,1,4
"(120, 'Building types total')",1,1,0,0,1,1,4
"(100, 'Blocks of flats, two-room flat')",0,1,1,0,1,1,4


In [61]:
res_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38 entries, (130, 'Building types total') to (200, 'Building types total')
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   p       38 non-null     int64
 1   d       38 non-null     int64
 2   q       38 non-null     int64
 3   P       38 non-null     int64
 4   D       38 non-null     int64
 5   Q       38 non-null     int64
 6   s       38 non-null     int64
dtypes: int64(7)
memory usage: 2.4+ KB


In [11]:
res_df = pd.read_csv('res_df.csv',index_col=[0])
res_df

,p,d,q,P,D,Q,s
"(130, 'Building types total')",2,1,0,2,1,1,4
"(130, 'Blocks of flats total')",2,1,0,2,1,1,4
"(100, 'Blocks of flats, three-room flat+')",1,1,0,0,1,1,4
"(120, 'Building types total')",1,1,0,0,1,1,4
"(100, 'Blocks of flats, two-room flat')",0,1,1,0,1,1,4
"(120, 'Blocks of flats, three-room flat+')",0,1,1,0,1,1,4
"(100, 'Blocks of flats, one-room flat')",0,1,1,0,1,1,4
"(120, 'Blocks of flats, one-room flat')",0,1,1,0,1,1,4
"(120, 'Blocks of flats total')",0,1,1,0,1,1,4
"(120, 'Blocks of flats, two-room flat')",0,1,1,0,1,1,4
